In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/austingrant/CS/Austin_chess/myenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/austingrant/CS/Austin_chess/myenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/austingrant/CS/Austin_chess/myenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739,

Kaggle Data set
https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations

In [2]:
df = pd.read_csv('archive/chessData.csv')

In [3]:
print(df.head())

                                                 FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26


In [4]:
def convert_to_index(pos: str) -> int:
    if pos == "-":
        return 0
    file_map = {value: key for key, value in enumerate("abcdefgh")}
    col = file_map[pos[0]]
    row = int(pos[1]) - 1
    # print(f"col: {col}, row: {row}")
    return 1 << (row * 8) + col


In [5]:

def convert_fen(fen: str):
    piece_map = {value: 0 for key, value in enumerate("KQRBNPkqrbnp")}
    # print(f"Piece_Map: {piece_map}")
    ranks = fen.split("/")
    # print(f"ranks: {ranks}")
    curr_rank = 8
    while curr_rank > 0:
        r = ranks[8-curr_rank]
        file_index = 0
        # print(f"rank: {curr_rank-1} | {r}")
        for i in range(len(r)):
            c = r[i]
            if file_index >= 8:
                states = r[i:].split(" ")[1:]
                # print(f"States: {states}")
                piece_map["moves"] = 0 if states[0] == 'w' else 1
                piece_map["castle"] = [0 if c == "-" else 1 for c in states[1]] # KQkq
                piece_map["en_passant"] = convert_to_index(states[2])
                piece_map["half_clock"] = int(states[3])
                piece_map["full_clock"] = int(states[4])
                return piece_map
            else:
                if c.isdigit():
                    file_index += int(c)
                else:
                    # print(f"c: {c}")
                    piece_map[c] += (1 << ((curr_rank * 8) + file_index))
                    file_index += 1
        curr_rank -= 1
    return piece_map
                


In [7]:
df['longs'] = df['FEN'].apply(convert_fen)

In [67]:
print(df['longs'][0])

{'K': 4096, 'Q': 2048, 'R': 33024, 'B': 9216, 'N': 16896, 'P': 68735139840, 'k': 295147905179352825856, 'q': 147573952589676412928, 'r': 2379629985508532158464, 'b': 664082786653543858176, 'n': 1217485108864830406656, 'p': 18374686479671623680, 'moves': 1, 'castle': [1, 1, 1, 1], 'en_passant': 0, 'half_clock': 0, 'full_clock': 1}


In [69]:
extended_df = pd.json_normalize(df['longs'])
print(extended_df.columns)
extended_df['White_Castle_King'] = extended_df['castle'][0]
extended_df['White_Castle_Queen'] = extended_df['castle'][1]
extended_df['Black_Castle_King'] = extended_df['castle'][2]
extended_df['Black_Castle_Queen'] = extended_df['castle'][3]

Index(['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'moves',
       'castle', 'en_passant', 'half_clock', 'full_clock'],
      dtype='object')


In [80]:
d = [extended_df['castle'] != [1,1,1,1]]
print(d)

ValueError: ('Lengths must match to compare', (12958035,), (4,))

In [90]:

print(extended_df[mask].head)

In [72]:

extended_df['White_Castle_King'] = extended_df['castle'].apply(lambda l: l[0])
extended_df['White_Castle_Queen'] = extended_df['castle'].apply(lambda l: l[1])
extended_df['Black_Castle_King'] = extended_df['castle'].apply(lambda l: l[2])
extended_df['Black_Castle_Queen'] = extended_df['castle'].apply(lambda l: l[3])
print(extended_df.columns)

IndexError: list index out of range

In [3]:
b = list(zip([1, 2, 3], ['a', 'b', 'c']))
b.sort(key=lambda x:x[0], reverse=True)
print(b)

[(3, 'c'), (2, 'b'), (1, 'a')]
